# A minimal ALF run

In this bare-bones example we use the [pyALF](https://git.physik.uni-wuerzburg.de/ALF/pyALF) interface to run the canonical Hubbard model on a default configuration: a $6\times6$ square grid, with interaction strength $U=4$ and inverse temperature $\beta = 5$.  

Bellow we go through the steps for performing the simulation and outputting observables.

---

**1.** Import `ALF_source` and `Simulation` classes from the `py_alf` python module, which provide the interface with ALF:

In [1]:
from py_alf import ALF_source, Simulation  # Interface with ALF

**2.** Create an instance of `ALF_source`, downloading the ALF source code from the [ALF repository](https://git.physik.uni-wuerzburg.de/ALF), if `alf_dir` does not exist. Gets alf_dir from environment variable `$ALF_DIR`, or defaults to "./ALF", if not present.

In [2]:
alf_src = ALF_source(
    branch='196-write-parameters-to-hdf5-file', # TODO: Remove this after merging '196-write-parameters-to-hdf5-file' into master
)

Checking out branch 196-write-parameters-to-hdf5-file


**3.** Create an instance of `Simulation`, overwriting default parameters as desired:

In [3]:
sim = Simulation(
    alf_src,
    "Hubbard",                    # Name of Hamiltonian
    {                             # Dictionary overwriting default parameters
        "Lattice_type": "Square"
    },
    machine='GNU'  # Change to "intel", or "PGI" if gfortran is not installed
)

**4.** Compile ALF. The first time it will also download and compile HDF5, which could take $\sim$15 minutes.

In [4]:
sim.compile()

Compiling ALF... 
Done.


**5.** Perform the simulation as specified in `sim`:

In [5]:
sim.run()

Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square" for Monte Carlo run.
Resuming previous run.
Run /home/stafusa/ALF/ALF/Prog/ALF.out


**6.** Perform some simple analysis:

In [6]:
sim.analysis()

### Analyzing /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square ###
/home/stafusa/ALF/pyALF/Notebooks
Custom observables:
Scalar observables:
Ener_scal
Kin_scal
Part_scal
Pot_scal
Histogram observables:
Equal time observables:
Den_eq
Green_eq
SpinT_eq
SpinXY_eq
SpinZ_eq
Time displaced observables:
Den_tau
Green_tau
SpinT_tau
SpinXY_tau
SpinZ_tau


**7.** Read analysis results into a Pandas Dataframe with one row per simulation, containing parameters and observables:

In [7]:
obs = sim.get_obs()

/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square


In [8]:
obs

,ham_t,ham_chem,ham_u,ham_t2,ham_u2,ham_tperp,mz,continuous,model,lattice_type,...,SpinXY_tauK,SpinXY_tauK_err,SpinXY_tauR,SpinXY_tauR_err,SpinXY_tau_lattice,SpinZ_tauK,SpinZ_tauK_err,SpinZ_tauR,SpinZ_tauR_err,SpinZ_tau_lattice
/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square,1.0,0.0,0.1,1.0,0.1,1.0,1,0,b'Hubbare',b'Square',...,"[[0.5080266768413108, 0.37851260608102755, 0.5...","[[5.509438133168012e-05, 7.093431943390947e-05...","[[0.0007086319587675874, -0.07129490901080424,...","[[7.801742975744647e-06, 1.788119343962928e-05...","{'L1': [4.0, 0.0], 'L2': [0.0, 4.0], 'a1': [1....","[[0.5079988476165884, 0.3787679553894949, 0.50...","[[0.0003022149100057337, 0.0001183243160476130...","[[0.000750127148593184, -0.07133024751410687, ...","[[8.991861174531717e-05, 9.45125099143898e-05,...","{'L1': [4.0, 0.0], 'L2': [0.0, 4.0], 'a1': [1...."


$\bullet$ The internal energy of the system (and its error) are accessed by:

In [9]:
obs.iloc[0][['Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'Ener_scal_sign_err']]

Ener_scal0           -23.601812
Ener_scal0_err         0.000337
Ener_scal_sign              1.0
Ener_scal_sign_err          0.0
Name: /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object

$\bullet$ The simulation can be resumed by calling `sim.run()` again, increasing the precision of results:

In [10]:
sim.run()
sim.analysis()
obs = sim.get_obs()
obs.iloc[0][['Ener_scal0', 'Ener_scal0_err', 'Ener_scal_sign', 'Ener_scal_sign_err']]

Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square" for Monte Carlo run.
Resuming previous run.
Run /home/stafusa/ALF/ALF/Prog/ALF.out
### Analyzing /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square ###
/home/stafusa/ALF/pyALF/Notebooks
Custom observables:
Scalar observables:
Ener_scal
Kin_scal
Part_scal
Pot_scal
Histogram observables:
Equal time observables:
Den_eq
Green_eq
SpinT_eq
SpinXY_eq
SpinZ_eq
Time displaced observables:
Den_tau
Green_tau
SpinT_tau
SpinXY_tau
SpinZ_tau
/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square


Ener_scal0           -23.60205
Ener_scal0_err        0.000318
Ener_scal_sign             1.0
Ener_scal_sign_err         0.0
Name: /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Hubbard_Square, dtype: object